![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.


## Podstawy Spark SQL - UDF

Podobnie jak w Hive czy wielu bazach danych, Spark SQL ma możliwość definiowania funkcji użytkownika, ang. User Defined Functions (UDF). Funkcje te biorą wartość z kolumny i przekształcają ją w inną wartość. 

In [1]:
import pyspark
import pyspark.sql.functions as func
import pyspark.sql.types as types

spark = pyspark.sql.SparkSession.builder \
    .appName('udf') \
    .getOrCreate()

#sc = pyspark.SparkContext(appName='udf')
#sqlContext = pyspark.sql.SQLContext(sc)

Wygenerujmy najpierw dane:

In [2]:
import numpy as np
df = spark.createDataFrame([pyspark.sql.Row(kolumna=int(i)) for i in np.random.randint(0, 100, 100)])
df.printSchema()
df.show()

root
 |-- kolumna: long (nullable = true)

+-------+
|kolumna|
+-------+
|     44|
|     82|
|     22|
|     49|
|     50|
|     32|
|     59|
|     13|
|     37|
|     75|
|     59|
|     43|
|     84|
|     15|
|     74|
|     26|
|     52|
|     54|
|     60|
|      6|
+-------+
only showing top 20 rows



Do stworzenia funkcji używamy normalnej referencji do funkcji Python lub lambdy. Przykładowo, chcemy funkcję która zwróci klasyfikacje wartości:

In [3]:
def klasyfikuj(wartosc):
    return u'dużo' if wartosc > 50 else u'mało'

Powyższą funkcję należy teraz przekształcić w funkcje która działa na kolumnach. Robimy to używając funkcji `udf`:

In [4]:
klasyfikuj_udf = func.udf(klasyfikuj)

In [7]:
klasyfikuj_udf

function

Tej funkcji można już użyć na kolumnie DataFrame, np możemy dodać kolumnę:

In [8]:
df.withColumn('ile?', klasyfikuj_udf('kolumna')).show()

+-------+----+
|kolumna|ile?|
+-------+----+
|     44|mało|
|     82|dużo|
|     22|mało|
|     49|mało|
|     50|mało|
|     32|mało|
|     59|dużo|
|     13|mało|
|     37|mało|
|     75|dużo|
|     59|dużo|
|     43|mało|
|     84|dużo|
|     15|mało|
|     74|dużo|
|     26|mało|
|     52|dużo|
|     54|dużo|
|     60|dużo|
|      6|mało|
+-------+----+
only showing top 20 rows



Należu uważać z typami; jeżeli chcemy zwrócić inny typ niż tekstowy, należy przekazać tą informację w definicji funkcji UDF.

In [17]:
oblicz_udf = func.udf(lambda v: v + 123, returnType=types.IntegerType())

In [18]:
df.withColumn('oblicz', oblicz_udf('kolumna')).show()

+-------+------+
|kolumna|oblicz|
+-------+------+
|     44|   167|
|     82|   205|
|     22|   145|
|     49|   172|
|     50|   173|
|     32|   155|
|     59|   182|
|     13|   136|
|     37|   160|
|     75|   198|
|     59|   182|
|     43|   166|
|     84|   207|
|     15|   138|
|     74|   197|
|     26|   149|
|     52|   175|
|     54|   177|
|     60|   183|
|      6|   129|
+-------+------+
only showing top 20 rows



Aby wykorzystać funckje w zapytaniach SQL należy ją zarejestrować nieco inaczej, mianowicie:

In [19]:
spark.udf.register('klasyfikuj', klasyfikuj, returnType=types.StringType())

In [20]:
df.registerTempTable('df')
spark.sql('SELECT kolumna, klasyfikuj(kolumna) AS `ile?` FROM df').show()

+-------+----+
|kolumna|ile?|
+-------+----+
|     44|mało|
|     82|dużo|
|     22|mało|
|     49|mało|
|     50|mało|
|     32|mało|
|     59|dużo|
|     13|mało|
|     37|mało|
|     75|dużo|
|     59|dużo|
|     43|mało|
|     84|dużo|
|     15|mało|
|     74|dużo|
|     26|mało|
|     52|dużo|
|     54|dużo|
|     60|dużo|
|      6|mało|
+-------+----+
only showing top 20 rows



In [22]:
@func.udf(types.IntegerType())
def foo(x):
    return 2*x

In [23]:
df.withColumn('foo', foo('kolumna')).show()

+-------+---+
|kolumna|foo|
+-------+---+
|     44| 88|
|     82|164|
|     22| 44|
|     49| 98|
|     50|100|
|     32| 64|
|     59|118|
|     13| 26|
|     37| 74|
|     75|150|
|     59|118|
|     43| 86|
|     84|168|
|     15| 30|
|     74|148|
|     26| 52|
|     52|104|
|     54|108|
|     60|120|
|      6| 12|
+-------+---+
only showing top 20 rows



## Dostęp do JVM

Spark używa [py4j](https://www.py4j.org/) aby wykonywać komendy na JVM. Dostęp do klas mamy w PySpark nieco ułatwiony, mianowicie używamy `_jvm` ze `SparkContext` podając pełną nazwę klasy z pakietem:

In [24]:
s = spark.sparkContext._jvm.java.lang.String('tekst')
print(s, type(s))

tekst <class 'str'>


Jak widać, py4j konwertuje typy. Możemy też użyć bardziej skomplikowanych obiektów:

In [25]:
tokenizer = spark.sparkContext._jvm.java.util.StringTokenizer('Ala ma kota!')
print(tokenizer, type(tokenizer))

java.util.StringTokenizer@771f4555 <class 'py4j.java_gateway.JavaObject'>


In [27]:
while(tokenizer.hasMoreTokens()):
    print(tokenizer.nextToken())

Lub prościej:

In [28]:
p = spark.sparkContext._jvm.java.util.regex.Pattern.compile('\\s')
print(p, type(p))
l = p.split('Ala ma kota!')
print(l, type(l))
list(l)

\s <class 'py4j.java_gateway.JavaObject'>
[Ljava.lang.String;@624fd522 <class 'py4j.java_collections.JavaArray'>


['Ala', 'ma', 'kota!']

Oczywiście lepiej powyższą funkcjonalność wykorzystać do funkcji zwracających RDD lub DataFrame, które można użyć bezpośrednio w Pythonie.